In [0]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.models import Model
from keras.models import Sequential
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Dropout, Embedding, concatenate


from keras.models import Model

Using TensorFlow backend.


In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# !ls My Drive/Colab Notebooks/
# gdrive/My Drive/Colab Notebooks/

In [0]:
""" DEPRECIATED --> Using training and GLOVE data straight from Google Drive """


# Import data from local drive
# from google.colab import files
# uploaded = files.upload()

' DEPRECIATED --> Using training and GLOVE data straight from Google Drive '

In [0]:
""" DEPRECIATED --> Using training and GLOVE data straight from Google Drive """
## load local data into CSV
# import io
# df = pd.read_csv(io.BytesIO(uploaded['train.csv'])).fillna(" ")

## Get Glove embeddings
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove*.zip

' DEPRECIATED --> Using training and GLOVE data straight from Google Drive '

In [0]:
# Get data from Google Drive
df = pd.read_csv('gdrive/My Drive/Colab Notebooks/train.csv').fillna(" ")

In [0]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

# Replace NA values
df['comment_text'].fillna(' ')

# Separate input features and target
y = df.loc[:, "toxic"]
X = df.loc[:,'comment_text']


# split into train and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)


# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
non_toxic = X[X["toxic"]==0]
toxic = X[X["toxic"]==1]

# # upsample minority
# toxic_upsampled = resample(toxic,
#                           replace=True, # sample with replacement
#                           n_samples=len(non_toxic), # match number in majority class
#                           random_state=27) # reproducible results

# # combine majority and upsampled minority
# X_upsampled = pd.concat([non_toxic, toxic_upsampled])





# downsample majority
non_toxic_downsampled = resample(non_toxic,
                                replace = False, # sample without replacement
                                n_samples = len(toxic), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
X_downsampled = pd.concat([non_toxic_downsampled, toxic])

# Resampling for random order
X_downsampled = X_downsampled.sample(frac=1).reset_index(drop=True)


# AGAIN: Separate input features and target
y = X_downsampled.loc[:, "toxic"]
X = X_downsampled.loc[:,'comment_text']


'''
    QUESTION: Followed: https://towardsdatascience.com/methods-for-dealing-with-imbalanced-data-5b761be45a18
    for upsampling where it states to split before upsampling to avoid overfitting the data. After they split the data, they
    resample the minority class, and then concatenate the results to a single Dataframe. In their model examples,
    it seems as though they are training their models on the entirity of the dataset and resuing data that their model has
    already seen for the test results, which should result in overfitting, right?
    
    So once I have the balanced upsampled dataset, I've resplit the dataset in training and testing

'''

# split into train and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)




num_non_toxic = X_downsampled.loc[X_downsampled["toxic"] == 0]
num_toxic = X_downsampled.loc[X_downsampled["toxic"] == 1]


print("Class Distribution across 2 classes:")
print(len(num_non_toxic),len(num_toxic))


max_features=100000
maxlen=150
embed_size=300


Class Distribution across 2 classes:
11393 11393


In [0]:
"""

    Tokenize split dataset (X_train, X_test)
    USE FOR TRAINING MODELS INDIVIDUALLY

"""

# Vectorize text + Prepare GloVe Embedding
tokenizer = text.Tokenizer(num_words=max_features, lower=True)
tokenizer.fit_on_texts(list(X_train))

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)


In [0]:
"""
  LOADING AND INDEXING GLOVE WORD EMBEDDINGS (300-D)
"""

embeddings_index = {}
with open('gdrive/My Drive/Colab Notebooks/glove.6B.300d.txt', encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

word_index = tokenizer.word_index
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
"""

    Tokenize the entire dataset
    USE FOR CROSS VALIDATION 

"""

# Vectorize text + Prepare GloVe Embedding
tokenizer = text.Tokenizer(num_words=max_features, lower=True)
tokenizer.fit_on_texts(list(X))

import pickle


with open('gdrive/My Drive/Colab Notebooks/text_tokenizer.pickle', 'wb') as f:
    pickle.dump(tokenizer, f)


X = tokenizer.texts_to_sequences(X)
X = sequence.pad_sequences(X, maxlen=maxlen)

In [0]:
""" LSTM MODEL WITH ATTENTION LAYER"""


from keras.layers import Input, Flatten, Activation, multiply, TimeDistributed, RepeatVector, Permute, Lambda
from keras import backend as K

units = 32
max_length = 150
vocab_size = embedding_matrix.shape[0]
embedding_size = embedding_matrix.shape[1]


_input = Input(shape=[max_length], dtype='int32')

# get the embedding layer
embedded = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_size,
        input_length=max_length,
        trainable=False,
        weights=[embedding_matrix]
    )(_input)

# Bidirectional(LSTM(128, return_sequences=True))
# activations = LSTM(units, return_sequences=True)(embedded)
# Bidirectional(LSTM(64, return_sequences=True))
activations = Bidirectional(LSTM(32, return_sequences=True))(embedded)


# compute importance for each step
attention = TimeDistributed(Dense(1, activation='tanh'))(activations) 
attention = Flatten()(attention)
attention = Activation('softmax')(attention)
# adjusting vector shapes for multiplication
attention = RepeatVector(64)(attention)
attention = Permute([2, 1])(attention)

# apply the attention

sent_representation = multiply([activations, attention])

sent_representation = Lambda(lambda xin: K.sum(xin, axis=1))(sent_representation)

probabilities = Dense(1, activation='sigmoid')(sent_representation)

model = Model(input=_input, output=probabilities)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
# len(y)//10
non_toxic_samples, non_toxic_labels = pd.DataFrame(X[0:11393]), pd.DataFrame(y[0:11393])
toxic_samples, toxic_labels, = pd.DataFrame(y[11393:]),pd.DataFrame(y[11393:])

# non_toxic_samples
# non_tox = non_toxic_samples.merge(non_toxic_labels)
# tox = toxic_samples.merge(toxic_labels)

In [0]:
predictions = model.predict(X_test, verbose=1)

In [0]:
def map_to_bool(predictions):
  """ Convert array of probabilities to array of boolean values to easily score """
  # for pred in predictions:
  # Map probabilities from continous values to either 1 or 0
  for index, prob in enumerate(predictions):
    # change likely toxic probability to 1
    if prob[0] >= 0.5:
      predictions[index] = 1
    else:
      predictions[index] = 0

  return predictions

In [0]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score
def model_metrics(predictions, y_test):
  """ Create easy to read object containing model evaluation scores """
  scores = {}
  predictions = map_to_bool(predictions)

  scores["f1_score"] = f1_score(y_test, predictions)
  scores["precision_score"] = precision_score(y_test, predictions)    
  scores["recall_score"] = recall_score(y_test, predictions)
  scores["roc_auc_score"] = roc_auc_score(y_test, predictions)
  
  return scores
  # LSTM w/ Attn
  # print("LSTM w/ Attn")
  # print(f"recall_score: {recall_sc}")
  # print(f"precision_score: {precision_sc}")
  # print(f"f1_score: {f1_sc}")


In [0]:
print("LSTM w/ Attn")
print(model_metrics(predictions,y_test))

In [0]:
"LSTM w/ CNN"
from keras.layers import GlobalMaxPooling1D

from sklearn.model_selection import cross_validate
from keras.wrappers.scikit_learn import KerasClassifier


# Grid Search model
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def bi_lstm_cnn():

  max_length = 150
  vocab_size = embedding_matrix.shape[0]
  embedding_size = embedding_matrix.shape[1]

  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size,input_length=max_length, weights=[embedding_matrix], trainable=False))
  model.add(Bidirectional(LSTM(128, return_sequences=True)))
  model.add(Conv1D(128, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform'))
  model.add(GlobalMaxPooling1D())

  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
  return model




lstm_cnn_model = KerasClassifier(build_fn=bi_lstm_cnn, epochs=10, batch_size=32, verbose=1)


scores = cross_validate(lstm_cnn_model, X, y, cv=10, scoring=['precision', 'f1', 'recall'], return_estimator=True)





# epochs = [15]
# batch_size = [16]

# param_grid = dict(epochs=epochs, batch_size=batch_size)

# bi_lstm_cnn_model = KerasClassifier(build_fn=bi_lstm_cnn, verbose=1)
# grid = GridSearchCV(estimator=bi_lstm_cnn_model, param_grid=param_grid, scoring = ['precision', 'f1', 'recall'], refit='f1', cv=5, verbose=1)
# grid_result = grid.fit(X, y)













Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
20507/20507 [==============================] - 276s 13ms/step - loss: 0.2659 - acc: 0.8857
Epoch 2/10
20507/20507 [==============================] - 271s 13ms/step - loss: 0.2020 - acc: 0.9172
Epoch 3/10
20507/20507 [==============================] - 274s 13ms/step - loss: 0.1609 - acc: 0.9354
Epoch 4/10
20507/20507 [==============================] - 272s 13ms/step - loss: 0.1174 - acc: 0.9540
Epoch 5/10
20507/20507 [==============================] - 273s 13ms/step - loss: 0.0716 - acc: 0.9740
Epoch 6/10
20507/20507 [==============================] - 274s 13ms/step - loss: 0.0363 - acc: 0.9880
Epoch 7/10
20507/20507 [==============================] - 273s 13ms/step - loss: 0.0225 - acc: 0.9929
Epoch 8/10
20507/20507 [==============================] - 273s 13ms/step - loss: 0.0119 - acc: 0.9967
Epoch 9/10
20507/20507 [==============================] - 274s 13ms/step - lo

In [0]:
models = scores["estimator"]


def save_model(model, split_num, model_name_type):
  """ Save each model used per cross validation split in Google Drive """
    # saving model
    model_filename = f'{model_name_type}_split_{split_num}.h5'
    # saving weights
    model.save(f"/content/gdrive/My Drive/Colab Notebooks/{model_filename}", overwrite=True)


for split_num, model in enumerate(models):
  # print(model.model)
  save_model(model.model, split_num+1, "lstm_cnn")


# !ls gdrive/My\ Drive/Colab\ Notebooks/

# def load_model():
#     # loading model
#     model = model_from_json(open('model_architecture.json').read())
#     model.load_weights('model_weights.h5')
#     model.compile(loss='categorical_crossentropy', optimizer='adam')
#     return model


In [0]:
scores

In [0]:
print("LSTM w/CNN")
predictions = model.predict(X[11000:12000], verbose=1)
# print(predictions[0:10])
print(model_metrics(predictions,y[11000:12000]))

LSTM w/CNN
1000/1000 [==============================] - 6s 6ms/step
{'f1_score': 0.8866498740554156, 'precision_score': 0.9041095890410958, 'recall_score': 0.8698517298187809, 'roc_auc_score': 0.8636790455709682}


In [0]:
from keras.layers import MaxPooling1D, Input, GlobalMaxPool1D, MaxPooling2D, Flatten,TimeDistributed

max_length = 150
vocab_size = embedding_matrix.shape[0]
embedding_size = 300

inp = Input(shape=(max_length,))
x = Embedding(vocab_size, embed_size, weights=[embedding_matrix], trainable=False)(inp)
x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = Flatten()(x)
x = Dense(1, activation="sigmoid")(x)
lstm_model = Model(inputs=inp, outputs=x)
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X_train[0:10], y_train[0:10], batch_size=16, epochs=10, verbose=1)


Epoch 1/10
10/10 [==============================] - 3s 330ms/step - loss: 0.6975 - acc: 0.6000
Epoch 2/10
10/10 [==============================] - 1s 53ms/step - loss: 0.5202 - acc: 0.9000
Epoch 3/10
10/10 [==============================] - 1s 53ms/step - loss: 0.4009 - acc: 1.0000
Epoch 4/10
10/10 [==============================] - 1s 56ms/step - loss: 0.3422 - acc: 1.0000
Epoch 5/10
10/10 [==============================] - 1s 55ms/step - loss: 0.2716 - acc: 1.0000
Epoch 6/10
10/10 [==============================] - 1s 55ms/step - loss: 0.2404 - acc: 1.0000
Epoch 7/10
10/10 [==============================] - 1s 54ms/step - loss: 0.1907 - acc: 1.0000
Epoch 8/10
10/10 [==============================] - 1s 54ms/step - loss: 0.1500 - acc: 1.0000
Epoch 9/10
10/10 [==============================] - 1s 54ms/step - loss: 0.1165 - acc: 1.0000
Epoch 10/10
10/10 [==============================] - 0s 46ms/step - loss: 0.0789 - acc: 1.0000


In [0]:
print("bi-LSTM No max pooling")
predictions = lstm_model.predict(X_test[0:100], verbose=1)
# print(predictions)
predictions = map_to_bool(predictions)
# print(predictions)
print(model_metrics(predictions,y_test[0:100]))




bi-LSTM No max pooling
100/100 [==============================] - 2s 18ms/step
{'f1_score': 0.5777777777777778, 'precision_score': 0.5306122448979592, 'recall_score': 0.6341463414634146, 'roc_auc_score': 0.6221579164944192}


In [0]:
""" LSTM MODEL -- With Max Pooling """

from keras.layers import MaxPooling1D, Input, GlobalMaxPool1D, MaxPooling2D
from sklearn.model_selection import cross_validate
from keras.wrappers.scikit_learn import KerasClassifier

def create_bi_lstm():

  max_length = 150
  vocab_size = embedding_matrix.shape[0]
  embedding_size = 300

  inp = Input(shape=(max_length,))
  x = Embedding(vocab_size, embed_size, weights=[embedding_matrix], trainable=False)(inp)
  x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
  x = Flatten()(x)
  x = Dense(1, activation="sigmoid")(x)
  lstm_model = Model(inputs=inp, outputs=x)
  lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return lstm_model


bi_lstm_model = KerasClassifier(build_fn=create_bi_lstm, epochs=10, batch_size=32, verbose=1)

scores = cross_validate(bi_lstm_model, X, y, cv=10, scoring=['accuracy', 'precision', 'f1', 'recall'])

# Grid Search Bi-LSTM model
# from sklearn.model_selection import GridSearchCV
# from keras.wrappers.scikit_learn import KerasClassifier

# epochs = [10, 20]
# batch_size = [16, 32]
# lstm_cells= [64, 128]
# is_trainable = [False, True]

# param_grid = dict(epochs=epochs, batch_size=batch_size, lstm_cells=lstm_cells, is_trainable=is_trainable)

# bi_lstm_model = KerasClassifier(build_fn=create_bi_lstm, verbose=1)
# grid = GridSearchCV(estimator=bi_lstm_model, param_grid=param_grid, cv=10, verbose=1)
# grid_result = grid.fit(X_train, y_train)

Epoch 1/2
9/9 [==============================] - 3s 381ms/step - loss: 0.8096 - acc: 0.0000e+00
Epoch 2/2
1/1 [==============================] - 1s 1s/step


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/2
9/9 [==============================] - 4s 420ms/step - loss: 0.8847 - acc: 0.0000e+00
Epoch 2/2
1/1 [==============================] - 1s 1s/step
Epoch 1/2
9/9 [==============================] - 4s 460ms/step - loss: 0.6830 - acc: 0.6667
Epoch 2/2
1/1 [==============================] - 1s 1s/step
Epoch 1/2
9/9 [==============================] - 4s 490ms/step - loss: 0.6561 - acc: 0.8889
Epoch 2/2
1/1 [==============================] - 2s 2s/step
Epoch 1/2
9/9 [==============================] - 5s 526ms/step - loss: 0.5669 - acc: 1.0000
Epoch 2/2
1/1 [==============================] - 2s 2s/step
Epoch 1/2
9/9 [==============================] - 5s 555ms/step - loss: 0.4793 - acc: 1.0000
Epoch 2/2
1/1 [==============================] - 2s 2s/step
Epoch 1/2
9/9 [==============================] - 5s 599ms/step - loss: 0.7763 - acc: 0.0000e+00
Epoch 2/2
1/1 [==============================] - 2s 2s/step
Epoch 1/2
9/9 [==============================] - 6s 624ms/step - loss: 0.6090 - 

In [0]:
from keras.layers import Input, Flatten, Activation, multiply, TimeDistributed, RepeatVector, Permute, Lambda
from keras import backend as K

from sklearn.model_selection import cross_validate
from keras.wrappers.scikit_learn import KerasClassifier

"""
  Bi-LSTM w/ Attn and K-fold cross validation
"""

def create_attn_lstm():


  units = 32
  max_length = 150
  vocab_size = embedding_matrix.shape[0]
  embedding_size = embedding_matrix.shape[1]


  _input = Input(shape=[max_length], dtype='int32')

  # get the embedding layer
  embedded = Embedding(
          input_dim=vocab_size,
          output_dim=embedding_size,
          input_length=max_length,
          trainable=False,
          weights=[embedding_matrix]
      )(_input)

  # Bidirectional(LSTM(128, return_sequences=True))
  # activations = LSTM(units, return_sequences=True)(embedded)
  # Bidirectional(LSTM(64, return_sequences=True))
  activations = Bidirectional(LSTM(32, return_sequences=True))(embedded)


  # compute importance for each step
  attention = TimeDistributed(Dense(1, activation='tanh'))(activations) 
  attention = Flatten()(attention)
  attention = Activation('softmax')(attention)
  # adjusting vector shapes for multiplication
  attention = RepeatVector(64)(attention)
  attention = Permute([2, 1])(attention)

  # apply the attention
  sent_representation = multiply([activations, attention])

  sent_representation = Lambda(lambda xin: K.sum(xin, axis=1))(sent_representation)

  probabilities = Dense(1, activation='sigmoid')(sent_representation)

  model = Model(input=_input, output=probabilities)
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model


attn_lstm_model = KerasClassifier(build_fn=create_attn_lstm, epochs=10, batch_size=32, verbose=1)

scores = cross_validate(attn_lstm_model, X, y, cv=10, scoring=['precision', 'f1', 'recall'],return_estimator=True)


print(scores)














Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`




Epoch 1/10
20507/20507 [==============================] - 371s 18ms/step - loss: 0.3483 - acc: 0.8481
Epoch 2/10
20507/20507 [==============================] - 367s 18ms/step - loss: 0.2546 - acc: 0.8947
Epoch 3/10
20507/20507 [==============================] - 372s 18ms/step - loss: 0.2273 - acc: 0.9073
Epoch 4/10
20507/20507 [==============================] - 372s 18ms/step - loss: 0.2062 - acc: 0.9166
Epoch 5/10
20507/20507 [==============================] - 370s 18ms/step - loss: 0.1969 - acc: 0.9204
Epoch 6/10
20507/20507 [==============================] - 368s 18ms/step - loss: 0.1893 - acc: 0.9257
Epoch 7/10
20507/20507 [==============================] - 374s 18ms/step - loss: 0.1674 - acc: 0.9352
Epoch 8/10
20507/20507 [==============================] - 367s 18ms/step - loss: 0.1477 - acc: 0.9434
Epoch 9/10
20507/20507 [==============================] - 371s 18ms/step - loss: 0.1305 - acc: 0.9508
Epoch 10/10
2279/2279 [==============================] - 16s 7ms/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/10
20507/20507 [==============================] - 374s 18ms/step - loss: 0.3432 - acc: 0.8540
Epoch 2/10
20507/20507 [==============================] - 375s 18ms/step - loss: 0.2702 - acc: 0.8869
Epoch 3/10
20507/20507 [==============================] - 370s 18ms/step - loss: 0.2443 - acc: 0.8997
Epoch 4/10
20507/20507 [==============================] - 375s 18ms/step - loss: 0.2232 - acc: 0.9081
Epoch 5/10
20507/20507 [==============================] - 371s 18ms/step - loss: 0.2109 - acc: 0.9154
Epoch 6/10
20507/20507 [==============================] - 376s 18ms/step - loss: 0.1849 - acc: 0.9252
Epoch 7/10
20507/20507 [==============================] - 375s 18ms/step - loss: 0.1673 - acc: 0.9358
Epoch 8/10
20507/20507 [==============================] - 367s 18ms/step - loss: 0.1490 - acc: 0.9412
Epoch 9/10
20507/20507 [==============================] - 372s 18ms/step - loss: 0.1321 - acc: 0.9483
Epoch 10/10
2279/2279 [==============================] - 16s 7ms/step


...

Epoch 1/10
20508/20508 [==============================] - 368s 18ms/step - loss: 0.3475 - acc: 0.8506
Epoch 2/10
20508/20508 [==============================] - 357s 17ms/step - loss: 0.2863 - acc: 0.8811
Epoch 3/10
20508/20508 [==============================] - 351s 17ms/step - loss: 0.2447 - acc: 0.9014
Epoch 4/10
20508/20508 [==============================] - 362s 18ms/step - loss: 0.2196 - acc: 0.9116
Epoch 5/10
20508/20508 [==============================] - 347s 17ms/step - loss: 0.2007 - acc: 0.9208
Epoch 6/10
20508/20508 [==============================] - 355s 17ms/step - loss: 0.1849 - acc: 0.9272
Epoch 7/10
20508/20508 [==============================] - 361s 18ms/step - loss: 0.1649 - acc: 0.9367
Epoch 8/10
20508/20508 [==============================] - 351s 17ms/step - loss: 0.1494 - acc: 0.9441
Epoch 9/10
20508/20508 [==============================] - 347s 17ms/step - loss: 0.1317 - acc: 0.9514
Epoch 10/10
2278/2278 [==============================] - 16s 7ms/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/10
20508/20508 [==============================] - 337s 16ms/step - loss: 0.3327 - acc: 0.8541
Epoch 2/10
20508/20508 [==============================] - 332s 16ms/step - loss: 0.2421 - acc: 0.9016
Epoch 3/10
20508/20508 [==============================] - 338s 16ms/step - loss: 0.2244 - acc: 0.9094
Epoch 4/10
20508/20508 [==============================] - 331s 16ms/step - loss: 0.2217 - acc: 0.9075
Epoch 5/10
20508/20508 [==============================] - 328s 16ms/step - loss: 0.1949 - acc: 0.9212
Epoch 6/10
20508/20508 [==============================] - 333s 16ms/step - loss: 0.1806 - acc: 0.9287
Epoch 7/10
20508/20508 [==============================] - 333s 16ms/step - loss: 0.1762 - acc: 0.9300
Epoch 8/10
20508/20508 [==============================] - 336s 16ms/step - loss: 0.1821 - acc: 0.9274
Epoch 9/10
20508/20508 [==============================] - 336s 16ms/step - loss: 0.1538 - acc: 0.9393
Epoch 10/10
2278/2278 [==============================] - 16s 7ms/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/10
20508/20508 [==============================] - 343s 17ms/step - loss: 0.3513 - acc: 0.8457
Epoch 2/10
20508/20508 [==============================] - 342s 17ms/step - loss: 0.2623 - acc: 0.8930
Epoch 3/10
20508/20508 [==============================] - 345s 17ms/step - loss: 0.2380 - acc: 0.9033
Epoch 4/10
20508/20508 [==============================] - 344s 17ms/step - loss: 0.2192 - acc: 0.9095
Epoch 5/10
20508/20508 [==============================] - 342s 17ms/step - loss: 0.2088 - acc: 0.9137
Epoch 6/10
20508/20508 [==============================] - 343s 17ms/step - loss: 0.1990 - acc: 0.9200
Epoch 7/10
20508/20508 [==============================] - 346s 17ms/step - loss: 0.1733 - acc: 0.9309
Epoch 8/10
20508/20508 [==============================] - 345s 17ms/step - loss: 0.1536 - acc: 0.9391
Epoch 9/10
20508/20508 [==============================] - 346s 17ms/step - loss: 0.1345 - acc: 0.9487
Epoch 10/10
2278/2278 [==============================] - 16s 7ms/step
{'fit_time':

In [0]:

models = scores["estimator"]


def save_model(model, split_num, model_name_type):
    # saving model
    model_filename = f'{model_name_type}_split_{split_num}.h5'
    # saving weights
    model.save(f"/content/gdrive/My Drive/Colab Notebooks/{model_filename}", overwrite=True)


for split_num, model in enumerate(models):
  # print(model.model)
  save_model(model.model, split_num+1, "lstm_attn")



In [0]:
print("LSTM with Max Pooling:")
predictions = lstm_model.predict(X_test, verbose=1)
print(model_metrics(predictions,y_test))

LSTM with Max Pooling:


ValueError: ignored

In [0]:
""" SIMPLE LSTM -- Using as baseline """

""" Dimensional issue with Bidirectional LSTM, Needs GlobalMaxPooling after Bi-LSTM layer """


"""
# From Farhad:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm = Bidirectional(LSTM(100, recurrent_dropout=0.1))(embedded_sequences)
preds = Dense(len(macronum), activation='softmax')(l_lstm)

"""

# inp = Input(shape=(max_length,))
# x = Embedding(vocab_size, embed_size, weights=[embedding_matrix])(inp)
# x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
# # x = GlobalMaxPool1D()(x)
# x = Dense(2, activation="sigmoid")(x)
# lstm_model = Model(inputs=inp, outputs=x)
# lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


"""
  FROM Medium: https://towardsdatascience.com/sentence-classification-using-bi-lstm-b74151ffa565

  # main model
  input = Input(shape=(max_len,))
  model = Embedding(vocab_size,100,weights=[embedding_matrix],input_length=max_len)(input)
  model =  Bidirectional (LSTM (100,return_sequences=True,dropout=0.50),merge_mode='concat')(model)
  model = TimeDistributed(Dense(100,activation='relu'))(model)
  model = Flatten()(model)
  model = Dense(100,activation='relu')(model)
  output = Dense(3,activation='softmax')(model)
  model = Model(input,output)
  model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

"""

"Simple LSTM"
from keras.layers import GlobalMaxPooling1D

from sklearn.model_selection import cross_validate
from keras.wrappers.scikit_learn import KerasClassifier


# Grid Search model
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_lstm():

  max_length = 150
  vocab_size = embedding_matrix.shape[0]
  embedding_size = 300


  simple_lstm = Sequential()
  simple_lstm.add(Embedding(vocab_size, embed_size, weights=[embedding_matrix], trainable=False))
  simple_lstm.add(LSTM(128))
  simple_lstm.add(Dense(1, activation='sigmoid'))
  simple_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return simple_lstm





simple_lstm = KerasClassifier(build_fn=create_lstm, epochs=10, batch_size=32, verbose=1)


scores = cross_validate(simple_lstm, X, y, cv=10, scoring=['precision', 'f1', 'recall'], return_estimator=True)


scores


# batch_size = 16
# simple_lstm.fit(X_train, y_train, batch_size=batch_size, epochs=10, verbose=1)












Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
20507/20507 [==============================] - 114s 6ms/step - loss: 0.3131 - acc: 0.8678
Epoch 2/10
20507/20507 [==============================] - 110s 5ms/step - loss: 0.2303 - acc: 0.9048
Epoch 3/10
20507/20507 [==============================] - 109s 5ms/step - loss: 0.1977 - acc: 0.9207
Epoch 4/10
20507/20507 [==============================] - 108s 5ms/step - loss: 0.1681 - acc: 0.9327
Epoch 5/10
20507/20507 [==============================] - 109s 5ms/step - loss: 0.1366 - acc: 0.9453
Epoch 6/10
20507/20507 [==============================] - 109s 5ms/step - loss: 0.1067 - acc: 0.9607
Epoch 7/10
20507/20507 [==============================] - 108s 5ms/step - loss: 0.1051 - acc: 0.9600
Epoch 8/10
20507/20507 [==============================] - 109s 5ms/step - loss: 0.0616 - acc: 0.9787
Epoch 9/10
20507/20507 [==============================] - 108s 5ms/step - loss: 0.040

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
20507/20507 [==============================] - 110s 5ms/step - loss: 0.3084 - acc: 0.8689
Epoch 2/10
20507/20507 [==============================] - 109s 5ms/step - loss: 0.2467 - acc: 0.8985
Epoch 3/10
20507/20507 [==============================] - 109s 5ms/step - loss: 0.2043 - acc: 0.9145
Epoch 4/10
20507/20507 [==============================] - 109s 5ms/step - loss: 0.1776 - acc: 0.9276
Epoch 5/10
20507/20507 [==============================] - 108s 5ms/step - loss: 0.1525 - acc: 0.9396
Epoch 6/10
20507/20507 [==============================] - 109s 5ms/step - loss: 0.1198 - acc: 0.9531
Epoch 7/10
20507/20507 [==============================] - 109s 5ms/step - loss: 0.0911 - acc: 0.9656
Epoch 8/10
20507/20507 [==============================] - 107s 5ms/step - loss: 0.0602 - acc: 0.9784
Epoch 9/10
20507/20507 [==============================] - 109s 5ms/step - loss: 0.0403 - acc: 0.9869
Epoch 10/10
2279/2279 [==============================] - 2s 1ms/step
Epoch 1/10
20507/20507

{'estimator': (<keras.wrappers.scikit_learn.KerasClassifier at 0x7fd1ec43c748>,
  <keras.wrappers.scikit_learn.KerasClassifier at 0x7fd174a70e48>),
 'fit_time': array([1102.0127604 , 1088.00492692, 1083.28889608, 1088.39755249,
        1109.52912617, 1128.83751106, 1103.56242037, 1102.50659418,
        1071.64536595, 1078.35892987]),
 'score_time': array([2.38141727, 2.47667694, 2.72192764, 2.62916875, 2.79354215,
        2.65654922, 2.76153636, 2.94455934, 2.87201262, 2.92859626]),
 'test_f1': array([0.        , 0.        , 0.        , 0.        , 0.01277955,
        0.94368482, 0.92572506, 0.9458584 , 0.95528548, 0.95910441]),
 'test_precision': array([0.        , 0.        , 0.        , 0.        , 0.00643087,
        1.        , 1.        , 1.        , 1.        , 1.        ]),
 'test_recall': array([0.        , 0.        , 0.        , 0.        , 1.        ,
        0.89337429, 0.86172081, 0.89727831, 0.9143986 , 0.9214223 ])}

In [0]:
models = scores["estimator"]
def save_model(model, split_num, model_name_type):
    # saving model
    model_filename = f'{model_name_type}_split_{split_num}.h5'
    # saving weights
    model.save(f"/content/gdrive/My Drive/Colab Notebooks/{model_filename}", overwrite=True)


for split_num, model in enumerate(models):
  # print(model.model)
  save_model(model.model, split_num+1, "simple_lstm")

In [0]:
print("Simple LSTM:")
predictions = simple_lstm.predict(X_test, verbose=1)
print(model_metrics(predictions,y_test))

Simple LSTM:
5697/5697 [==============================] - 8s 1ms/step
{'f1_score': 0.9064493267186392, 'precision_score': 0.9116179615110478, 'recall_score': 0.9013389711064129, 'roc_auc_score': 0.9072976772286174}


In [0]:
""" CNN Model """
from keras import regularizers

def create_cnn_model(num_filters, kernal_sizes, neurons, is_trainable):

  #model parameters
  max_length = 150
  weight_decay = 1e-4

  #CNN architecture
  print("training CNN ...")
  cnn_model = Sequential()
  cnn_model.add(Embedding(vocab_size, embedding_size,
            weights=[embedding_matrix], input_length=max_length, trainable=is_trainable))
  cnn_model.add(Conv1D(num_filters, kernal_sizes, activation='relu', padding='same'))
  cnn_model.add(MaxPooling1D(2))
  cnn_model.add(Conv1D(num_filters, kernal_sizes, activation='relu', padding='same'))
  cnn_model.add(GlobalMaxPooling1D())
  cnn_model.add(Dense(neurons, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
  cnn_model.add(Dense(1, activation='sigmoid'))

  cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return cnn_model


In [0]:
# Parameter Tuning for CNN
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

epochs = [150,300]
batch_size = [16]
num_filters = [32, 64, 128]
kernal_sizes = [2,3,4]
is_trainable = [False, True]
neurons = [16, 32, 64]


# Test SGD optimizer ---> https://arxiv.org/pdf/1705.08292.pdf

param_grid = dict(epochs=epochs, batch_size=batch_size, kernal_sizes= kernal_sizes, num_filters=num_filters, is_trainable=is_trainable,neurons=neurons)

cnn_model = KerasClassifier(build_fn=create_cnn_model, verbose=1)
grid = GridSearchCV(estimator=cnn_model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)


Using TensorFlow backend.


NameError: ignored

In [0]:
""" Train CNN w/o parameter tuning"""

  #training params
  batch_size = 256 
  num_epochs = 8 
  #model parameters
  num_filters = 64 
  max_length = 150
  weight_decay = 1e-4

  #CNN architecture
  print("training CNN ...")
  cnn_model = Sequential()
  cnn_model.add(Embedding(vocab_size, embedding_size,
            weights=[embedding_matrix], input_length=max_length, trainable=False))
  cnn_model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
  cnn_model.add(MaxPooling1D(2))
  cnn_model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
  cnn_model.add(GlobalMaxPooling1D())
  cnn_model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
  cnn_model.add(Dense(1, activation='sigmoid'))

  cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

from keras.callbacks import EarlyStopping

#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

#model training
hist = cnn_model.fit(X_train, y_train, batch_size=batch_size, epochs=200, callbacks=callbacks_list, verbose=0)

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


In [0]:
print("CNN")
predictions = cnn_model.predict(X_test, verbose=1)
print(model_metrics(predictions,y_test))

CNN
5697/5697 [==============================] - 1s 196us/step
{'f1_score': 0.8960787761561455, 'precision_score': 0.8943488943488943, 'recall_score': 0.897815362931642, 'roc_auc_score': 0.89626689797509}


In [0]:
print(grid_result.best_params_)
print(grid_result.best_score_)